In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
import time
import random
import datetime
import re
#import pickle

## blog URL

In [4]:
def get_blogs(html):
    results = []
    urls = []
    soup = BeautifulSoup(html, 'lxml')
    lis = soup.find_all('li', attrs={'class':'sh_blog_top', 'id':re.compile('sp_blog')})
    for li in lis:
        title='NA'
        naver_url = 'NA'
#        pub = 'NA'
#        date = 'NA'
        title = li.find('a', attrs={'class':re.compile(r'_sp_each_title')}).get('title').strip()
        if li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).text:
            naver_url = li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).get('href').strip()
#        pub = li.find('span', attrs={'class':'_sp_each_source'}).text.strip()
#        date = li.find('span', attrs={'class':'bar'}).next_sibling.strip()
        results.append([title, naver_url])
        urls.append([naver_url])
        print(title, naver_url)
        
    return urls

## real data URL

In [5]:
def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url   

## Title

In [6]:
def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

## Datetime

In [7]:
def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

## Text

In [8]:
def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts

##  naver URL 구성
* where = post(블로그)<br/>
* query = <br/>
* st = sim(관련도순), date(최신순) <br/>
* date_from = <br/>
* date_to = <br/>
* date_option = 0(전체), 2,3,4,5,6,7(1일,1주,1개월,6개월,1년), 8(직접입력) <br/>
* srchby = all(전체), title(제목)<br/>

## selenium으로 blog url 불러오기

In [9]:
from tqdm import tqdm_notebook

In [44]:
driver = webdriver.Chrome(executable_path=r'C:\Users\sormd\chromedriver.exe')

query = 'skt 누구'

#pickle_name = q+'_'+startDate+'_'+endDate+'.p'
total_url = []
for page in tqdm_notebook(range(1,101)):
    #sleep_time = random.random()*2
    time.sleep(1)
    
    start = page*10 + 1
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=post&query=' +query+ '&start='+ str(start)
    driver.get(url)
    html = driver.page_source
    if re.search(r'검색결과가 없습니다',html): # 네이버기사 한번에 20page까지 가능
        break
        
    results = get_blogs(html)
    #print(page, results, len(results))
    total_url.extend(results)
driver.close()

#pickle.dump(total_results.open(pickle_name,'wb'))


인공지능스피커 SKT 누구 미니 (NUGU mini) 연결 방법 https://blog.naver.com/sanghyo1026?Redirect=Log&logNo=221320882069
SKT NUGU 인공지능 스피커 누구캔들 3달째 사용중! https://blog.naver.com/reeyoung82?Redirect=Log&logNo=221359106388
최근 지름샷 (feat. 두피마사지기, SKT 누구 미니 등) https://blog.naver.com/kecodak?Redirect=Log&logNo=221403585449
인공지능 스피커 SKT 누구 미니 - sk nugu mini 사용 후기 https://blog.naver.com/eunj_79?Redirect=Log&logNo=221310826697
SKT 누구 캔들 아이와 함께한 한달 사용기 https://blog.naver.com/dgd1005?Redirect=Log&logNo=221410919933
아리아 AI와 무드등이 탑재된 SKT 누구 캔들 (NUGU candle) 인공지능 스피커 https://blog.naver.com/moimoi1357?Redirect=Log&logNo=221333274672
SKT 누구 미니 (NUGU mini) 귀염 터지는 인공지능 스피커 https://blog.naver.com/warm_today?Redirect=Log&logNo=221086871943
[인공지능 스피커] SKT 누구(NUGU) 캔들 B tv 연결 설정 및 사용기 http://hare.kr/221410935280
SKT 누구(NUGU candle) 인공지능스피커와 조명의 만남! https://blog.naver.com/sweet_buzzi?Redirect=Log&logNo=221323063255
NUGU 스피커 skt 누구와 말 배우기 https://blog.naver.com/xodndi1010?Redirect=Log&logNo=221333321292
SKT 인공지능형 스피커 누구 미니(NUGU 

본격 1가정 1비서 시대, 인공지능 스피커 (SKT 누구, 아마존 에코, 구글 홈) 분석 https://blog.naver.com/hyunjs1015?Redirect=Log&logNo=220876164456
번개장터 skt 누구 블루투스 스피커 https://blog.naver.com/kdh2001119?Redirect=Log&logNo=221458250389
SKT NUGU "누구" https://blog.naver.com/oklim88?Redirect=Log&logNo=220887453737
누구 캔들 AI 인공지능스피커 SKT누구 무드등 멋지게 연출하자 https://cdmanii.com/6553?category=428841
skt 누구 사용기, 60% 할인된 성능 http://gargoil.egloos.com/5877057
인공지능 스피커 SKT NUGU MINI와 함께해요. https://blog.naver.com/lawkapp?Redirect=Log&logNo=221284071668
SKT 인공지능 스피커, 누구! https://blog.naver.com/hanbok_yedan?Redirect=Log&logNo=221304790509
SKT누구캔들, 집에서 혼자놀기 지존되겠어~ https://blog.naver.com/ib1004i?Redirect=Log&logNo=221386283208
SKT 누구[NUGU] 개봉기 및 솔직한 리뷰 https://blog.naver.com/showtime_2am?Redirect=Log&logNo=221099015007
[SKT NUGU] 'SKT 누구 스피커' 현재까지 서비스 업데이트 한눈에 알아보기.. 및 개인적인 후기 https://blog.naver.com/t2313?Redirect=Log&logNo=220945254834
SKT 누구덕분에 일찍 시작한 크리스마스 이브~ https://blog.naver.com/hyolheej?Redirect=Log&logNo=220893673608
SKT 누구(NUGU) 갖

SKT 누구(NUGU) https://blog.naver.com/lsh2133?Redirect=Log&logNo=220894887822
한국형 에코 - SKT NUGU 누구 ... 3개월후 운명은? https://rontie.blog.me/220823909493
SKT 인공지능 누구(NUGU), 피자와 치킨 배달 메뉴가 늘어났다 https://vaders.blog.me/220886100004
이놈의 SKT 인생! 무료에 할인에 좋다 좋아!ㅎㅎㅎㅎ https://vaders.blog.me/220904454129
skt 누구 인공지능 음성인식 스피커! 너무 유용해요 https://blog.naver.com/limkun0542?Redirect=Log&logNo=220924497923
SKT '누구 미니 (NUGU mini)' 오해 풀기!! (Solve the Misunderstanding) https://blog.naver.com/567658?Redirect=Log&logNo=221097466993
skt NUGU 누구 캔들 사용법 리뷰 https://blog.naver.com/dbsrk777?Redirect=Log&logNo=221392848270
SKT 새로운 인공지능 스피커 누구 캔들(NUGU candle) 사용기 http://az-it.kr/221329417005
SKT - 누구(NUGU)에게나 생길법한 이야기_워킹맘 편 https://uyeobbeul.blog.me/220982845075
SKT 누구(NUGU), 지금 사도될까? https://blog.naver.com/asdf132456?Redirect=Log&logNo=220822074294
skt 누구 연결 NUGU 디바이스 와이파이 연결이 안되는 경우 해결방법 https://blog.naver.com/logo-gune?Redirect=Log&logNo=220987994963
SKT 누구(NUGU)로 치킨, 피자 주문할 수 있다!? 누구 사용후기 https://blog.naver.com/sunshine

skt 아리아 누구(NUGU) 스피커 https://blog.naver.com/mjk50210?Redirect=Log&logNo=220877624254
[행복TV]플랫폼으로 진화하는 SKT 음성AI "누구" https://www.only1happy.co.kr/38
SKT, 누구 고객과 가을 소리 나눈다…꿀사운드 코리아 개최 https://blog.naver.com/jinmun03?Redirect=Log&logNo=221382154254
SKT vs. 진에어 레딧 반응 (2018.06.24) https://blog.naver.com/overjoyed5?Redirect=Log&logNo=221305816063
똑똑한 인공지능 블루투스 스피커 - SKT 누구 https://blog.naver.com/dadailuda?Redirect=Log&logNo=220936387089
SKT 누구 mini 출시?? https://blog.naver.com/teno3d?Redirect=Log&logNo=221072088483
스마트홈을 위한 SKT 내가고른패키지 어때? https://dutni72.blog.me/221379908748
SKT '누구'·KT '기가지니' 동시에 써보니…단순 정보 제공 뛰어나지만 언어인식률은 아쉬워 http://blog.daum.net/dragonkim/15467393
2월 티데이(t day) - 빕스(vips) 40% 할인,most 모스트 주유할인, 롯데월드 55% 할인,스타벅스,t맵 택시 할인, skt 티멤버십 혜택 https://blog.naver.com/jijili?Redirect=Log&logNo=221456354745
SKT 인공지능 스피커 누구? 어땠으면 좋겠어? https://blog.naver.com/2yunsik?Redirect=Log&logNo=221403527642
세계최초로 5G 상용화 시작한 SKT 초(超)시대가 우리의 생활이 된다! https://blog.naver.com/k_159500?Redirect=Log&logNo=2

SKT멤버십 T멤버십으로 하와이여행준비하기 ~ https://blog.naver.com/jjungmin1417?Redirect=Log&logNo=221177897949
SKT해외로밍 연말여행 괌 가려면 괌사이판패스 먼저 챙겨 https://blog.naver.com/kwoncharm_e?Redirect=Log&logNo=221404745702
skt 누구 1주년 기념 이벤트 당첨 선물 NUGU mini형님 NUGU https://y07366.blog.me/221125831760
아웃백 SKT회원 누구나 T멤버십 차감없이 15%추가할인! 이번주 데이트, 소개팅은 아웃백에서~ https://blog.naver.com/dingdong_no1?Redirect=Log&logNo=220340173523
5G로 달라질 나의 라이프 :: SKT 퀀텀 NUGU(누구) 인공지능 자율주행차 https://blog.naver.com/wlaud3443?Redirect=Log&logNo=220945352345
2019 서울모터쇼 SKT 5G 자율주행 자동차 어서 타고 싶어요 https://blog.naver.com/k_159500?Redirect=Log&logNo=221503066370
SKT vs. bbq 레딧 반응 (2018.07.25) https://blog.naver.com/overjoyed5?Redirect=Log&logNo=221326491821
티월드다이렉트 오늘도착, SKT에서 제안하는 휴대폰 구매 꿀팁 https://blog.naver.com/moyeu119?Redirect=Log&logNo=221388895345
인공지능 스피커 추천할 만 하네! SKT NUGU candle https://blog.naver.com/koizz?Redirect=Log&logNo=221327475914
SKT NUGU MINI, '인공지능 AI스피커로, 다양한 기능 활용하기' https://blog.naver.com/yzk12272?Redirect=Log&logNo=221284266319

SKT 고객만족도 1위는 바로 T멤버십에서부터 https://blog.naver.com/foren00n?Redirect=Log&logNo=221190354645
포켓몬과 만난 SKT '누구'…5000개 한정 14만원에 시판 https://blog.naver.com/yl3901?Redirect=Log&logNo=221022036520
skt 멤버쉽 혜택 꼭 체크해보기 https://madbaseb.tistory.com/101
번개장터 [SKT] 인공지능 스피커 NUGU 누구/음성인식 스피커 https://blog.naver.com/js7743en?Redirect=Log&logNo=221465236467
여행할인 이젠! SKT 제휴 " T*tourplus " http://blog.daum.net/sshdeni/48
SKT 가정용 미세먼지 측정기 에어큐브 ( 윤슬 파파 ) https://blog.naver.com/nhj0812?Redirect=Log&logNo=221001063240
광주 전남 SKT텔레콤 0캠퍼스 대학생이라면 누구나~ https://blog.naver.com/0205008?Redirect=Log&logNo=221391529131
갤럭시S9 SKT 금액및 기능보고 비교까지해보자 ~ https://blog.naver.com/lohasfam?Redirect=Log&logNo=221485394429
[App소개] 신개념 모바일 무료신문 : 스마트데일리(갤탭&SKT Only) https://luke1011.blog.me/70098907664
SKT 누구미니(NUGU mini) 핫스팟 설정방법 https://blog.naver.com/ydd2d?Redirect=Log&logNo=221411943983
SKT T로밍과 함께 가는 크리스마스 일본 여행지 추천 https://blog.naver.com/misspark365?Redirect=Log&logNo=221417210524
[엔터] SKT 음악 사업 진출 기사 https://blog.naver.com/jako

In [6]:
#for i in total_results:
#    print(i[0])

https://blog.naver.com/trendhan?Redirect=Log&logNo=221432584681
https://blog.naver.com/sanghyo1026?Redirect=Log&logNo=221320882069
https://blog.naver.com/kecodak?Redirect=Log&logNo=221403585449
https://blog.naver.com/reeyoung82?Redirect=Log&logNo=221359106388
https://blog.naver.com/moimoi1357?Redirect=Log&logNo=221333274672
https://blog.naver.com/eunj_79?Redirect=Log&logNo=221310826697
https://blog.naver.com/dgd1005?Redirect=Log&logNo=221410919933
http://hare.kr/221410935280
https://blog.naver.com/abc_bridge?Redirect=Log&logNo=221362344275
https://blog.naver.com/warm_today?Redirect=Log&logNo=221086871943
https://blog.naver.com/xodndi1010?Redirect=Log&logNo=221333321292
https://blog.naver.com/cjasdm1?Redirect=Log&logNo=221086437436
https://blog.naver.com/seon_me?Redirect=Log&logNo=221364038245
https://blog.naver.com/trendhan?Redirect=Log&logNo=221431155280
https://blog.naver.com/80may11?Redirect=Log&logNo=221363530398
https://blog.naver.com/lidaki?Redirect=Log&logNo=221253631833
https:/

## Get real_url, title, datetime, text

In [45]:
with open('skt누구1000.txt', 'w', encoding = 'utf8') as f:
    title_list = []
    publish_datetime_list = []
    text_list = []
    
    for full_url in tqdm_notebook(total_url):
        # print(full_url[0])
        
        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(full_url[0])
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)
        title_list.append(title)

        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        publish_datetime_list.append(publish_datetime)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)
        text_list.append(texts)

#         print('url: ', full_url)
#         print('real_url: ', real_url, '\n')
#         print('post title: ', title, '\n')
#         print('publish date time: ', publish_datetime, '\n')
#         print("texts: ", texts)
#         print("\n-----------------------------------\n")
        
        #content = ' '.join(texts)
        f.write(texts+'\n') #결과를 text 파일에 저장합니다.

        time.sleep(1)

## dataframe.to_csv

In [46]:
import pandas as pd
df = pd.DataFrame(data={'title': title_list, 
                        'publish_datetime': publish_datetime_list,
                        'text': text_list, 
                        'url': total_url})

In [47]:
df = df[['title', 'publish_datetime', 'text', 'url']]
df.head()

,title,publish_datetime,text,url
0,인공지능스피커 SKT 누구 미니 (NUGU mini) 연결 방법,2018-07-17 17:00:00,저희 집에는 인공지능스피커인 카카오 미니와 클로버가 있어요.그리고 얼마 전 SKT ...,[https://blog.naver.com/sanghyo1026?Redirect=L...
1,SKT NUGU 인공지능 스피커 누구캔들 3달째 사용중!,2018-09-14 19:32:00,세달 전에 우리집에 온 인공지능 스피커 누구캔들!밤마다 잠자리를 은은하게 밝혀주며힐...,[https://blog.naver.com/reeyoung82?Redirect=Lo...
2,"최근 지름샷 (feat. 두피마사지기, SKT 누구 미니 등)",2018-11-22 07:40:00,지난달 말부터 블랙프라이데이라고 여기저기서 난리더니 여전히 다양한 쇼핑 이벤트가 매...,[https://blog.naver.com/kecodak?Redirect=Log&l...
3,인공지능 스피커 SKT 누구 미니 - sk nugu mini 사용 후기,2018-07-03 07:00:00,"\n인공지능(AI), IOT가 많은 전자제품들에 적용되어 출시되고 있어요~보일러에서...",[https://blog.naver.com/eunj_79?Redirect=Log&l...
4,SKT 누구 캔들 아이와 함께한 한달 사용기,2018-12-02 22:50:00,매일 아침 날씨도 알려주고미세먼지도 말해주는 친절한 누구캔들벌써 한달째 사용하고 있...,[https://blog.naver.com/dgd1005?Redirect=Log&l...


In [49]:
len(df)

1000

In [50]:
df.to_csv('./skt_nugu1000.csv',sep=',',encoding='UTF-8')

In [51]:
pd.read_csv('./skt_nugu1000.csv',index_col=0)

,title,publish_datetime,text,url
0,인공지능스피커 SKT 누구 미니 (NUGU mini) 연결 방법,2018-07-17 17:00:00,저희 집에는 인공지능스피커인 카카오 미니와 클로버가 있어요.그리고 얼마 전 SKT ...,['https://blog.naver.com/sanghyo1026?Redirect=...
1,SKT NUGU 인공지능 스피커 누구캔들 3달째 사용중!,2018-09-14 19:32:00,세달 전에 우리집에 온 인공지능 스피커 누구캔들!밤마다 잠자리를 은은하게 밝혀주며힐...,['https://blog.naver.com/reeyoung82?Redirect=L...
2,"최근 지름샷 (feat. 두피마사지기, SKT 누구 미니 등)",2018-11-22 07:40:00,지난달 말부터 블랙프라이데이라고 여기저기서 난리더니 여전히 다양한 쇼핑 이벤트가 매...,['https://blog.naver.com/kecodak?Redirect=Log&...
3,인공지능 스피커 SKT 누구 미니 - sk nugu mini 사용 후기,2018-07-03 07:00:00,"\n인공지능(AI), IOT가 많은 전자제품들에 적용되어 출시되고 있어요~보일러에서...",['https://blog.naver.com/eunj_79?Redirect=Log&...
4,SKT 누구 캔들 아이와 함께한 한달 사용기,2018-12-02 22:50:00,매일 아침 날씨도 알려주고미세먼지도 말해주는 친절한 누구캔들벌써 한달째 사용하고 있...,['https://blog.naver.com/dgd1005?Redirect=Log&...
5,아리아 AI와 무드등이 탑재된 SKT 누구 캔들 (NUGU candle) 인공지능 스피커,2018-08-06 02:29:00,SKT 누구 인공지능 스피커를 사용한 뒤 아침 알람으로 잠에서 깨어난 뒤 일기예보나...,['https://blog.naver.com/moimoi1357?Redirect=L...
6,SKT 누구 미니 (NUGU mini) 귀염 터지는 인공지능 스피커,2017-09-01 18:00:00,이번에 SKT에서 새롭게 출시된 누구 미니 (NUGU mini)를 만나 볼 수 있었...,['https://blog.naver.com/warm_today?Redirect=L...
7,[인공지능 스피커] SKT 누구(NUGU) 캔들 B tv 연결 설정 및 사용기,2018-12-02 23:15:00,이전 누구 캔들 개봉 및 세팅을 하면서 전반적으로 소비자의 이야기를 듣고 개선한 ...,['http://hare.kr/221410935280']
8,SKT 누구(NUGU candle) 인공지능스피커와 조명의 만남!,2018-07-20 18:41:00,+SKT 누구(NUGU candle)AI 인공지능스피커와 조명의 만남 ! 신박해\n...,['https://blog.naver.com/sweet_buzzi?Redirect=...
9,NUGU 스피커 skt 누구와 말 배우기,2018-08-06 08:15:00,동물과 소통의 시도 중인 밍이는 외할머니 댁에 놀러 가서 동내를 다니며 처음 본 강...,['https://blog.naver.com/xodndi1010?Redirect=L...
